In [15]:
# Mrityunjay kumar #
# A Convolutional Network implementation example using TensorFlow library.
# This example is using the MNIST database of handwritten digits
# (http://yann.lecun.com/exdb/mnist/)


In [16]:
# Import MINST data

import tensorflow as tf
import numpy as np

tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()

import input_data
mnist = input_data.read_data_sets("MNIST/", one_hot=True)



Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [17]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 20

In [18]:
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

In [19]:
# tf Graph input
x = tf.compat.v1.placeholder(tf.compat.v1.float32, [None, n_input])
y = tf.compat.v1.placeholder(tf.compat.v1.float32, [None, n_classes])
keep_prob = tf.compat.v1.placeholder(tf.compat.v1.float32) #dropout (keep probability)

In [20]:
# Create model
def conv2d(img, w, b):
    return tf.compat.v1.nn.relu(tf.compat.v1.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b))

def max_pool(img, k):
    return tf.compat.v1.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.compat.v1.reshape(_X, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout
    conv1 = tf.compat.v1.nn.dropout(conv1, _dropout)

    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    conv2 = tf.compat.v1.nn.dropout(conv2, _dropout)

    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.compat.v1.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.compat.v1.nn.relu(tf.compat.v1.add(tf.compat.v1.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.compat.v1.nn.dropout(dense1, _dropout) # Apply Dropout

    # Output, class prediction
    out = tf.compat.v1.add(tf.compat.v1.matmul(dense1, _weights['out']), _biases['out'])
    return out

In [21]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.compat.v1.Variable(tf.compat.v1.random_normal([5, 5, 1, 32])), 
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.compat.v1.Variable(tf.compat.v1.random_normal([5, 5, 32, 64])), 
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.compat.v1.Variable(tf.compat.v1.random_normal([7*7*64, 1024])), 
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.compat.v1.Variable(tf.compat.v1.random_normal([1024, n_classes])) 
}

biases = {
    'bc1': tf.compat.v1.Variable(tf.compat.v1.random_normal([32])),
    'bc2': tf.compat.v1.Variable(tf.compat.v1.random_normal([64])),
    'bd1': tf.compat.v1.Variable(tf.compat.v1.random_normal([1024])),
    'out': tf.compat.v1.Variable(tf.compat.v1.random_normal([n_classes]))
}

In [22]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

In [23]:
# Define loss and optimizer
cost = tf.compat.v1.reduce_mean(tf.compat.v1.nn.softmax_cross_entropy_with_logits(logits =pred, labels=y))


optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [24]:
# Evaluate model
correct_pred = tf.compat.v1.equal(tf.compat.v1.argmax(pred,1), tf.compat.v1.argmax(y,1))
accuracy = tf.compat.v1.reduce_mean(tf.compat.v1.cast(correct_pred, tf.compat.v1.float32))

In [25]:
# Initializing the variables
init = tf.compat.v1.initialize_all_variables()

In [26]:
# Launch the graph
with tf.compat.v1.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    # Calculate accuracy for 256 mnist test images
    print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], 
                                                             y: mnist.test.labels[:256], 
                                                             keep_prob: 1.}))

Iter 2560, Minibatch Loss= 11219.332031, Training Accuracy= 0.38281
Iter 5120, Minibatch Loss= 4442.676270, Training Accuracy= 0.69531
Iter 7680, Minibatch Loss= 3588.013184, Training Accuracy= 0.72656
Iter 10240, Minibatch Loss= 3327.193604, Training Accuracy= 0.81250
Iter 12800, Minibatch Loss= 2574.712158, Training Accuracy= 0.79688
Iter 15360, Minibatch Loss= 2642.146484, Training Accuracy= 0.80469
Iter 17920, Minibatch Loss= 2110.542969, Training Accuracy= 0.82812
Iter 20480, Minibatch Loss= 1864.048462, Training Accuracy= 0.82812
Iter 23040, Minibatch Loss= 2441.991699, Training Accuracy= 0.80469
Iter 25600, Minibatch Loss= 2270.326172, Training Accuracy= 0.82812
Iter 28160, Minibatch Loss= 1562.228638, Training Accuracy= 0.85938
Iter 30720, Minibatch Loss= 1106.920166, Training Accuracy= 0.87500
Iter 33280, Minibatch Loss= 1780.205444, Training Accuracy= 0.84375
Iter 35840, Minibatch Loss= 1536.382080, Training Accuracy= 0.83594
Iter 38400, Minibatch Loss= 559.136353, Training A